## Introduction
Well Hello there! In this notebook, we'll go through predicting rainfall. A fairly simple, classification project. Perfect for newbies and intermediates. 

## Problem Statement

This tutorial takes a practical and coding-focused approach. We'll learn how to apply _logistic regression_ to a real-world dataset from [Kaggle](https://kaggle.com/datasets):

> **QUESTION**: The [Rain in Australia dataset](https://kaggle.com/jsphyg/weather-dataset-rattle-package) contains about 10 years of daily weather observations from numerous Australian weather stations. Here's a small sample from the dataset:
> 
> ![](https://i.imgur.com/5QNJvir.png)
>
> As a data scientist at the Bureau of Meteorology, you are tasked with creating a fully-automated system that can use today's weather data for a given location to predict whether it will rain at the location tomorrow. 
>

In this tutorial, we'll train a _logistic regression_ model using the Rain in Australia dataset to predict whether or not it will rain at a location tomorrow, using today's data. This is a _binary classification_ problem.
​
Let's install the `scikit-learn` library which we'll use to train our model.

In [1]:
!pip install scikit-learn --upgrade --quiet

## Downloading the Data

Just to make everything super easy for you, I'll be importing the dataset directly from the GitHub repo. I have already downloaded the dataset to the repo via the link: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package